In [2]:
!pip install efficientnet_pytorch
!pip install boto3
!pip install sagemaker
!pip install torch torchvision

In [3]:
import boto3
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, models
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from tqdm.auto import tqdm
import random
import numpy as np
import os
import copy
import time
import datetime
from PIL import Image
from io import BytesIO

In [4]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [5]:
# GPU 설정 및 시드 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

# 클래스 수 및 기타 설정
num_classes = 4
hyper_param_batch = 12
image_size = 224

In [6]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=4).to(device)

Loaded pretrained weights for efficientnet-b0


In [7]:
class S3Dataset(torch.utils.data.Dataset):
    def __init__(self, s3_client, bucket_name, prefix, transform=None):
        self.s3_client = s3_client
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.transform = transform
        self.files = self.retrieve_file_list()

    def retrieve_file_list(self):
        paginator = self.s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=self.bucket_name, Prefix=self.prefix)
        files = []
        for page in pages:
            for obj in page['Contents']:
                if obj['Key'].endswith('.jpg'):
                    files.append(obj['Key'])
        return files

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=self.files[idx])
        img = Image.open(BytesIO(obj['Body'].read()))
        img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)

        # 파일 경로에서 첫 번째 숫자만 레이블로 사용
        label = int(self.files[idx].split('/')[-2].split('.')[0])  # '0.양호' -> '0'
        return img, label

In [8]:
# S3 클라이언트 생성
s3_client = boto3.client('s3')
bucket_name = 'circleai'  # S3 버킷 이름
train_prefix = 'all_data/head/train/data/4.모낭홍반농포/'  # 훈련 데이터가 위치한 S3 경로
val_prefix = 'all_data/head/valid/data/4.모낭홍반농포/'  # 검증 데이터가 위치한 S3 경로

In [9]:
from torchvision import transforms
import torchvision.transforms.functional as TF

def random_transforms(tensor):
    # 사용자 정의 변환 함수
    if torch.rand(1) < 0.5:
        tensor = TF.adjust_gamma(tensor, gamma=0.9, gain=1)
    if torch.rand(1) < 0.5:
        tensor = TF.adjust_hue(tensor, hue_factor=-0.05)
    return tensor

transforms_train = transforms.Compose([
    #transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    #transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    #transforms.RandomGrayscale(p=0.25),
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, shear=15, scale=(0.8, 1.2)),
    #transforms.RandomPerspective(distortion_scale=0.25, p=0.5),
    transforms.Resize([int(300), int(300)], interpolation=4),
    transforms.ToTensor(),  # ToTensor는 여기에 위치
    transforms.Lambda(random_transforms),  # ToTensor 변환 후 사용자 정의 람다 변환 적용
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

#vaild_data Compose
transforms_val = transforms.Compose([
                                        transforms.Resize([int(300), int(300)], interpolation=4),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])

In [10]:
train_dataset = S3Dataset(s3_client, bucket_name, train_prefix, transform=transforms_train)
val_dataset = S3Dataset(s3_client, bucket_name, val_prefix, transform=transforms_val)

train_loader = DataLoader(train_dataset, batch_size=hyper_param_batch, persistent_workers=True, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=hyper_param_batch, persistent_workers=True, shuffle=False, num_workers=4)

In [11]:
best_acc = 0.0

In [12]:
from tqdm.auto import tqdm
global best_acc

def train_model(model, criterion, optimizer, scheduler, num_epochs, train_loader, val_loader):
    start_time = time.time()

    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 각 단계별로 학습 및 검증
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로 설정
            else:
                model.eval()   # 모델을 평가 모드로 설정

            running_loss = 0.0
            running_corrects = 0
            num_samples = 0

            # 데이터 반복
            with tqdm(train_loader if phase == 'train' else val_loader, desc=f"{phase} Epoch {epoch}") as t:
                for inputs, labels in t:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    optimizer.zero_grad()

                    # 순전파
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # 학습 단계에서만 역전파 + 최적화
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    num_samples += inputs.size(0)

                    epoch_loss = running_loss / num_samples
                    epoch_acc = running_corrects.double() / num_samples

                    # 진행 표시줄 업데이트
                    t.set_postfix(loss=epoch_loss, acc=epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            # 검증 단계에서 최고의 정확도를 갖는 모델 저장
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'현재 최고  검증률 : {best_acc}')


        print()

    return model, best_acc

In [13]:
# Criterion (손실 함수), Optimizer, Scheduler 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
epochs = 20

In [14]:
trained_model, best_accuracy = train_model(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)

model_path = f'erythema_pustules_{best_accuracy:.4f}.pt'
torch.save(trained_model, model_path)

# S3에 업로드할 때 파일 이름에 best_accuracy를 포함
s3_client.upload_file(model_path, bucket_name, f'all_data/head/pt_files/erythema_pustules_{best_accuracy:.4f}.pt')

Epoch 0/19
----------


train Epoch 0:   0%|          | 0/313 [00:00<?, ?it/s]

train Loss: 0.9026 Acc: 0.6381


val Epoch 0:   0%|          | 0/90 [00:00<?, ?it/s]

val Loss: 0.6929 Acc: 0.7150
현재 최고  검증률 : 0.7149532710280374

Epoch 1/19
----------


...

val Loss: 0.9608 Acc: 0.6804

Epoch 19/19
----------


train Epoch 19:   0%|          | 0/313 [00:00<?, ?it/s]

train Loss: 0.2462 Acc: 0.9077


val Epoch 19:   0%|          | 0/90 [00:00<?, ?it/s]

val Loss: 1.0072 Acc: 0.6832

